In [ ]:
import h5py
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import cv2
import sunpy.map
import astropy.units as u
from matplotlib.backends.backend_agg import FigureCanvasAgg
from itertools import cycle

input_dirs = [
    r'C:\Users\ebb04\bfield_filesT0.0',
    r'C:\Users\ebb04\bfield_filesT0.5',
    r'C:\Users\ebb04\bfield_filesT1.0',
    r'C:\Users\ebb04\bfield_filesT2.0'
]

phases = {
    "T0.0": [(4, 132), (132, 236), (236, 432)],
    "T0.5": [(4, 128), (128, 288), (288, 704)],
    "T1.0": [(4, 144), (144, 304), (304, 620)],
    "T2.0": [(4, 188), (188, 352), (352, 604)]
}

output_base_dir = r'C:\Users\ebb04\PILs_output_V7'

B0 = 0.13  # Tesla
L0 = 1.7e5  # meters
t0 = 24.9  # seconds
flux_conversion_factor = B0 * L0**2 * 1e8  # Maxwells
time_conversion_factor = t0 / 3600  # hours

In [ ]:
def read_h5_data(file_path):
    with h5py.File(file_path, 'r') as f:
        bx = f['bx'][:]
        by = f['by'][:]
        bz = f['bz'][:]
        x = f['x'][:]  
        y = f['y'][:]  
        dx = np.gradient(x)
        dy = np.gradient(y)
    return bx, by, bz, x, y, dx, dy

In [ ]:
def count_red_pixels(fig):
    canvas = FigureCanvasAgg(fig)
    canvas.draw()
    image = np.frombuffer(canvas.buffer_rgba(), dtype='uint8').reshape(fig.canvas.get_width_height()[::-1] + (4,))
    
    red_threshold = np.array([200, 0, 0, 255])
    red_mask = (image[:, :, 0] >= red_threshold[0]) & (image[:, :, 1] <= red_threshold[1]) & (image[:, :, 2] <= red_threshold[2]) & (image[:, :, 3] == red_threshold[3])
    red_pixel_count = np.sum(red_mask)
    
    return red_pixel_count

In [ ]:
def plot_filtered_contours(data, x, y, title, threshold, vmin=None, vmax=None):
    small_value_mask = (np.abs(data) < threshold)
    fig, ax = plt.subplots(figsize=(8, 8))
    extent = [x.min(), x.max(), y.min(), y.max()]
    img = ax.imshow(data, cmap='gray', origin='lower', vmin=-0.0001, vmax=0.0001, extent=extent)
    plt.colorbar(img, ax=ax, label='Bz $(B_0)$')
    ax.set_xlabel('x $(L_0)$')
    ax.set_ylabel('y $(L_0)$')
    ax.set_title(title)

    dx = np.gradient(x)
    dy = np.gradient(y)
    
    magnetogram = sunpy.map.Map((data, {
        'cunit1': 'arcsec',
        'cunit2': 'arcsec',
        'cdelt1': dx,
        'cdelt2': dy,
        'crval1': 0,
        'crval2': 0,
        'crpix1': 0.5 * (data.shape[1] + 1),
        'crpix2': 0.5 * (data.shape[0] + 1),
        'ctype1': 'HPLN-TAN',
        'ctype2': 'HPLT-TAN'
    }))
    
    cs = ax.contour(magnetogram.data, levels=[threshold], colors='red', extent=extent)
    filtered_contours = []

    for collection in cs.collections:
        for path in collection.get_paths():
            mask = np.zeros_like(small_value_mask, dtype=np.uint8)
            poly_verts = path.to_polygons()
            if len(poly_verts) > 0:
                poly = poly_verts[0]
                poly = np.array([[int(p[0]), int(p[1])] for p in poly])
                cv2.drawContours(mask, [poly], -1, 1, thickness=cv2.FILLED)
                if not np.any(np.logical_and(mask, small_value_mask)):
                    filtered_contours.append(poly)

    for contour in filtered_contours:
        ax.plot(contour[:, 0], contour[:, 1], color='red')

    red_pixel_count = count_red_pixels(fig)
    plt.close(fig)

    pixel_size = np.sqrt(dx**2 + dy**2)  # Using 1 for cdelt1 and cdelt2
    pil_length = red_pixel_count * pixel_size 

    return pil_length

In [ ]:
def dVdx_fft(V,dx):
    """
    In
      V : 2D Matrix
      dx : grid spacing of V matrix along x

    Out
      dVdx = invfft(fft(V[x,y])*kx)
      

    """
    ny, nx = np.shape(V)

    kx1d = 2.0*np.pi*np.fft.fftfreq(nx, d=dx)    

    Kx = 1j*np.tile(kx1d, (ny,1))
    
    dVdx = np.real(np.fft.ifft(np.fft.fft(V,axis=1)*Kx, axis=1))

    return(dVdx)


def dVdy_fft(V,dy):
    """
    In
      V : 2D Matrix
      dy : grid spacing of V matrix along x

    Out
      dVdy = invfft(fft(V[x,y])*ky)
      

    """
    ny, nx = np.shape(V)

    ky1d = 2.0*np.pi*np.fft.fftfreq(ny, d=dy)    

    Ky = np.transpose(1j*np.tile(ky1d, (nx,1)))
    
    dVdy = np.real(np.fft.ifft(np.fft.fft(V,axis=0)*Ky, axis=0))

    return(dVdy)


def CICCI(Bx,By,Bz, dx, dy):
    """
    The code find the contributions to the magnetic field from sources above, 
    below and from currents crossing a plain to the magnetic field for a peridic system.

    Based on Peter W. Schuck's idl code assosiated with his 2022 papper:

        Schuck, P. W., Linton, M. G., Knizhnik, K. J., & Leake,
        J. E. 2022, The Astrophysical Journal, 936, 94,
        doi: 10.3847/1538-4357/ac739a

    In
        Bx[iy,ix], By[iy,ix],Bz[iy,ix] : Magnetic field vetor in a plain
                                         index order as shown
        dx dy : the grid resolutoon

    Out
        T[iy,ix]     : Toroidal component
        PL[iy,ix]    : Poloidal in plain componets from below the plain
        PL_Z[iy,ix]  : Poloidal in perpendidular componets from below the plain
        PG[iy,ix]    : Poloidal in plain componets from abouv the plain
        PG_Z[iy,ix]  : Poloidal in perpendidular componets from abouv the plain
        BM[3]        : mean value of Bx, By, Bz
        BPL[iy,ix,3] : Bx,By,Bz from the Poloidal compinets form below
        BPG[iy,ix,3] : Bx,By,Bz from the Poloidal compinets form abow
        BT[iy,ix,2]  : Bx,By from troroidal compinets, from currents crossing the plain
        BP[iy,ix,3]  : Bx,By,Bz from the total Poloidal componets
        Jz[iy,ix]    : vertical currents crossing the plain
        JTz[iy,ix]   : toroidal vertical currents  assosiatd with poloidal fields
        JPz[iy,ix]   : poloidal vertical currents  assosiatd with troroidal fields
        dx dy        : the grid resolution (same as input)

    """

    nx, ny = np.shape(Bx)
    print(nx,ny)

    ky1d = 2.0*np.pi*np.fft.fftfreq(ny, d=dx)    
    Ky = np.tile(ky1d, (nx,1))

    kx1d = 2.0*np.pi*np.fft.fftfreq(nx, d=dy)    
    Kx = np.transpose(np.tile(kx1d, (ny,1)))

    k2 = np.square(Kx) + np.square(Ky)
    k2[0,0] = 0.0 # avoid singulaerty
    k = np.sqrt(k2) 
    
    BM = (np.mean(Bx), np.mean(By), np.mean(Bz))

    # Fourier transforms
    bxf = np.fft.fft2(Bx-BM[0])
    byf = np.fft.fft2(By-BM[1])
    bzf = np.fft.fft2(Bz-BM[2])

    # Poloidal function
    # P_>+P_<
    GTpLT = -bzf/k2
    GTpLT[0,0] = 0.0

    # surface divergence
    divf = 1j*Kx*bxf + 1j*Ky*byf
    #div  = np.fft.ifft(divf)
    #divr = dVdx_fft(Bx,dx)+dVdy_fft(By,dy) 

    # P_>-P_<
    GTmLT = divf/(np.abs(k)*k2)
    GTmLT[0,0] = 0.0

    PGF=(GTpLT+GTmLT)/2.0
    PLF=(GTpLT-GTmLT)/2.0
    PF=GTpLT
    
    #Toroidal function   
    Jz=dVdx_fft(By-BM[1],dx)-dVdy_fft(Bx-BM[0],dy) # Note Missing 4*Pi/c factor
    TF=-((1j*Kx*byf)-(1j*Ky*bxf))/k2 #(curl)
    TF[0,0]=0.0
    Jz=np.real(np.fft.ifft2(-k2*TF))

    # zderivatives of Poloidal functions
    PGF_Z =  k*PGF   
    PLF_Z = -k*PLF   
    PF_Z  = -k*PF   
    


    # reconstruct Toroidal (T) and Poloidal (PG,PL) Functions   
    T = np.real(np.fft.ifft2(TF))
    PG = np.real(np.fft.ifft2(PGF))
    PL = np.real(np.fft.ifft2(PLF))
    PG_Z = np.real(np.fft.ifft2(PGF_Z))
    PL_Z = np.real(np.fft.ifft2(PLF_Z))

    BT  = np.zeros((nx,ny,2))     # Toroidal field (Bx,By)
    BPG = np.zeros((nx,ny,3))     # Sources Chromosphere/Corona
    BPL = np.zeros((nx,ny,3))     # Sources Convection Zone
    BP  = np.zeros((nx,ny,3))     # Poloidal field
   
    # Toroidal Field
    BT[:,:,0]=np.real(np.fft.ifft2(-1j*Ky*TF))
    BT[:,:,1]=np.real(np.fft.ifft2(1j*Kx*TF))

    # Poloidal Field z-component
    BPG[:,:,2]=np.real(-np.fft.ifft2(k2*PGF))
    BPL[:,:,2]=np.real(-np.fft.ifft2(k2*PLF))
    BP[:,:,2]=np.real(-np.fft.ifft2(k2*PF))
   
    # Poloidal Field x-component
    BPG[:,:,0]=np.real(-np.fft.ifft2(1j*Kx*PGF_Z))
    BPL[:,:,0]=np.real(-np.fft.ifft2(1j*Kx*PLF_Z))
    BP[:,:,0]=np.real(-np.fft.ifft2(1j*Kx*PF_Z))

    # Poloidal Field y-component
    BPG[:,:,1]=np.real(-np.fft.ifft2(1j*Ky*PGF_Z))
    BPL[:,:,1]=np.real(-np.fft.ifft2(1j*Ky*PLF_Z))
    BP[:,:,1]=np.real(-np.fft.ifft2(1j*Ky*PF_Z))

    #J from Toroidal B  
    JPz=dVdx_fft(BT[:,:,1],dx)-dVdy_fft(BT[:,:,0],dy) # Note Missing 4*Pi/c factor
    #TF=-((1j*Kx*byf)-(1j*Ky*bxf))/k2 #(curl)
    #TF[0,0]=0.0
    #JPz=np.real(np.fft.ifft2(-k2*TF))

    #J from Poloidal B  
    JTz=dVdx_fft(BP[:,:,1],dx)-dVdy_fft(BP[:,:,0],dy) # Note Missing 4*Pi/c factor
    #TF=-((1j*Kx*byf)-(1j*Ky*bxf))/k2 #(curl)
    #TF[0,0]=0.0
    #JTz=np.real(np.fft.ifft2(-k2*TF))


    return(T,PL,PL_Z,PG,PG_Z,
            BM,BPL,BPG,BT,BP,Jz,JTz,JPz, dx,dy)

In [ ]:
def plot_1x3_contours(data_bpl, data_bz, data_bpg, x, y, time, twist, output_dir, file_index, threshold_bpl=1e-1, threshold_bz=2e-3, threshold_bpg=1e-1):
    fig, axs = plt.subplots(1, 3, figsize=(24, 8))
    extent = [x.min(), x.max(), y.min(), y.max()]

    titles = ['BPL', 'Bz', 'BPG']
    data_list = [data_bpl, data_bz, data_bpg]
    thresholds = [threshold_bpl, threshold_bz, threshold_bpg]

    for ax, data, title, threshold in zip(axs, data_list, titles, thresholds):
        ax.imshow(data, cmap='gray', origin='lower', vmin=data.min(), vmax=data.max(), extent=extent)
        plt.colorbar(ax.imshow(data, cmap='gray', origin='lower', vmin=data.min(), vmax=data.max(), extent=extent), ax=ax, label='Bz $(B_0)$')
        ax.set_xlabel('x $(L_0)$')
        ax.set_ylabel('y $(L_0)$')
        ax.set_title(f'{title} with $\\zeta$ {twist} at time {time} $t_0$')

        dx = np.gradient(x)
        dy = np.gradient(y)
        magnetogram = sunpy.map.Map((data, {
            'cunit1': 'arcsec',
            'cunit2': 'arcsec',
            'cdelt1': dx,
            'cdelt2': dy,
            'crval1': 0,
            'crval2': 0,
            'crpix1': 0.5 * (data.shape[1] + 1),
            'crpix2': 0.5 * (data.shape[0] + 1),
            'ctype1': 'HPLN-TAN',
            'ctype2': 'HPLT-TAN'
        }))
        
        cs = ax.contour(magnetogram.data, levels=[threshold], colors='red', extent=extent)
        filtered_contours = []

        for collection in cs.collections:
            for path in collection.get_paths():
                mask = np.zeros_like(data, dtype=np.uint8)
                poly_verts = path.to_polygons()
                if len(poly_verts) > 0:
                    poly = poly_verts[0]
                    poly = np.array([[int(p[0]), int(p[1])] for p in poly])
                    cv2.drawContours(mask, [poly], -1, 1, thickness=cv2.FILLED)
                    if not np.any(np.logical_and(mask, (np.abs(data) < threshold))):
                        filtered_contours.append(poly)

        for contour in filtered_contours:
            ax.plot(contour[:, 0], contour[:, 1], color='red')

    plt.tight_layout()
    plt.show()

    output_path = os.path.join(output_dir, f"{file_index:04d}.png")
    fig.savefig(output_path)
    print(f"Saved plot to {output_path}")

In [ ]:
def plot_2x2_pil_vs_time(pil_lengths, phases):
    fig, axs = plt.subplots(2, 2, figsize=(15, 15))
    twists = ['0.0', '0.5', '1.0', '2.0']
    titles = [f'Twist {twist}' for twist in twists]

    handles_labels = []

    for ax, twist, title in zip(axs.flat, twists, titles):
        if twist in pil_lengths:
            times = np.array(pil_lengths[twist]['time'])
            pil_bpl = np.array(pil_lengths[twist]['BPL'])
            pil_bz = np.array(pil_lengths[twist]['Bz'])
            pil_bpg = np.array(pil_lengths[twist]['BPG'])
            line_bpl = ax.plot(times, pil_bpl, label='BPL', color='blue')[0]
            line_bz = ax.plot(times, pil_bz, label='Bz', color='green')[0]
            line_bpg = ax.plot(times, pil_bpg, label='BPG', color='red')[0]
            ax.set_xlabel('Time ($t_0$)')
            ax.set_ylabel('PIL Length')
            ax.set_title(title)
            ax.grid(True)
            
            #phases are added here, but not necessary
            for (start, end), color in zip(phases[f'T{twist}'], cycle(['orange', 'purple', 'cyan'])):
                ax.axvspan(start, end, color=color, alpha=0.3)
            
            if not handles_labels:
                handles_labels.append((line_bpl, 'BPL'))
                handles_labels.append((line_bz, 'Bz'))
                handles_labels.append((line_bpg, 'BPG'))

    #I was having trouble with the legend being in the loop so I took it out.  you can fix as needed
    handles, labels = zip(*handles_labels)
    fig.legend(handles, labels, loc='upper center', ncol=3)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


In [ ]:
#same as with flux emergence
def calculate_pil_elongation_rate(pil_values, L0, t0):
    pil = np.array([value[1] for value in pil_values])
    pil_10th = np.percentile(pil, 10) 
    pil_90th = np.percentile(pil, 90) 
    time = np.array([value[0] for value in pil_values]) 

    idx_10th = np.where(pil <= pil_10th)[0][-1]
    idx_90th = np.where(pil >= pil_90th)[0][0]
    
    time_10th = time[idx_10th]
    time_90th = time[idx_90th]

    emerged_pil = (pil_90th - pil_10th) * L0
    pil_rate = emerged_pil / ((time_90th - time_10th) * t0)

    return pil_rate, emerged_pil

In [ ]:
def plot_elongation_rate_vs_max_pil(pil_lengths, L0, t0):
    fig, ax = plt.subplots(figsize=(10, 6))

    twists = ['0.0', '0.5', '1.0', '2.0']
    colors = ['purple', 'green', 'blue', 'red']
    markers = {'Bz': 'o', 'BPL': '^', 'BPG': 's'}
    labels = ['Bz', 'BPL', 'BPG']

    for twist, color in zip(twists, colors):
        for label in labels:
            pil_rate, max_pil = calculate_pil_elongation_rate(list(zip(pil_lengths[twist]['time'], pil_lengths[twist][label])), L0, t0)
            ax.scatter(max_pil, pil_rate, color=color, marker=markers[label], label=f'{label} Twist {twist}' if label == 'Bz' else "")

    ax.set_xlabel('Maximum PIL Length (m)')
    ax.set_ylabel('PIL Elongation Rate (m/hr)')
    ax.set_title('PIL Elongation Rate vs Maximum PIL Length') 
    #i used log scale to be consistent but this can be removed
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, which="both", ls="--")

    plt.tight_layout()
    plt.show()

In [ ]:
def EPILC(input_dirs, output_base_dir, threshold_bpl=1e-6, threshold_bz=1e-3, threshold_bpg=1e-6):
    pil_lengths = {twist: {'time': [], 'BPL': [], 'Bz': [], 'BPG': []} for twist in ['0.0', '0.5', '1.0', '2.0']}
    for input_dir in input_dirs:
        files_pattern = os.path.join(input_dir, 'bfield_*.h5')
        files = glob.glob(files_pattern)
        files.sort()
        twist = os.path.basename(input_dir).split('T')[-1]
        output_dir = os.path.join(output_base_dir, f'Twist{twist}')
        os.makedirs(output_dir, exist_ok=True)
        for i, file_path in enumerate(files):
            bx, by, bz, x, y, dx, dy = read_h5_data(file_path)
            T, PL, PL_Z, PG, PG_Z, BM, BPL, BPG, BT, BP, Jz, JTz, JPz, dx, dy = CICCI(bx, by, bz, dx, dy)
            time = (i + 1) * 4
            data_bz = bz
            print(f'Processing time step: {time}')
            pil_length_bpl = plot_filtered_contours(BPL[:, :, 2], x, y, f'BPL with $\\zeta$ {twist} at time {time} $t_0$', threshold_bpl)
            pil_length_bz = plot_filtered_contours(data_bz, x, y, f'Bz with $\\zeta$ {twist} at time {time} $t_0$', threshold_bz)
            pil_length_bpg = plot_filtered_contours(BPG[:, :, 2], x, y, f'BPG with $\\zeta$ {twist} at time {time} $t_0$', threshold_bpg)
            pil_lengths[twist]['time'].append(time)
            pil_lengths[twist]['BPL'].append(pil_length_bpl)
            pil_lengths[twist]['Bz'].append(pil_length_bz)
            pil_lengths[twist]['BPG'].append(pil_length_bpg)
            twist_dir = os.path.join(output_dir, f'{twist}')
            os.makedirs(twist_dir, exist_ok=True)
            plot_1x3_contours(BPL[:, :, 2], data_bz, BPG[:, :, 2], x, y, time, twist, twist_dir, i, threshold_bpl, threshold_bz, threshold_bpg)
    return pil_lengths

pil_lengths = EPILC(input_dirs, output_base_dir)

#i find it hepful to save to h5 so i can mess around with the data without having to rerun script
output_file = os.path.join(output_base_dir, 'pil_lengths.h5')
with h5py.File(output_file, 'w') as f:
    for twist, lengths in pil_lengths.items():
        grp = f.create_group(twist)
        for key, data in lengths.items():
            grp.create_dataset(key, data=data)

print(f'PIL lengths saved to {output_file}')

plot_2x2_pil_vs_time(pil_lengths, phases)
plot_elongation_rate_vs_max_pil(pil_lengths, L0, t0)